# Spoken Language Recognition Using Convolutional Neural Networks

_written by Joscha S. Rieber (Fraunhofer IAIS) in 2020_

## Environment

In [1]:
train = 'train'
test = 'test'

eng = 'english'
ger = 'german'
swe = 'swedish'

languages = [eng, ger, swe]
categories = [train, test]

dataset_root_path = '../data/'

sample_rate = 8000
image_width = 500
image_height = 128

## Import Libraries

In [15]:
import os
import librosa as lr
import librosa.display
#from librosa.display import waveplot
import numpy as np
from glob import glob
import matplotlib.pyplot as plt
import imageio
import warnings
from IPython.display import Audio

## Helper Functions

### Function for loading the audio file

In [3]:
def load_audio_file(audio_file_path):
    warnings.simplefilter('ignore', UserWarning)
    
    audio_segment, _ = lr.load(audio_file_path, sr=sample_rate)
    return audio_segment

    warnings.simplefilter('default', UserWarning)

### Function to repeat the audio segment to a length of 10 seconds

In [4]:
def fix_audio_segment_to_10_seconds(audio_segment):
    target_len = 10 * sample_rate
    audio_segment = np.concatenate([audio_segment]*3, axis=0)
    audio_segment = audio_segment[0:target_len]
    
    return audio_segment

### Function to generate the mel-scaled spectrogram from the 10-seconds long audio segment

In [5]:
def spectrogram(audio_segment):
    # Compute mel-scaled spectrogram image
    hl = audio_segment.shape[0] // image_width
    spec = lr.feature.melspectrogram(audio_segment, n_mels=image_height, hop_length=int(hl))

    # Logarithmic amplitudes
    image = lr.core.power_to_db(spec)

    # Convert to numpy matrix
    image_np = np.asmatrix(image)

    # Normalize and scale
    image_np_scaled_temp = (image_np - np.min(image_np))
    
    image_np_scaled = image_np_scaled_temp / np.max(image_np_scaled_temp)

    return image_np_scaled[:, 0:image_width]

### Function for converting to integer for image storing

In [6]:
def to_integer(image_float):
    # range (0,1) -> (0,255)
    image_float_255 = image_float * 255.
    
    # Convert to uint8 in range [0:255]
    image_int = image_float_255.astype(np.uint8)
    
    return image_int

### Determine all available audio files

In [7]:
audio_files = {}

for lang in languages:
    for category in categories:
        dataset_path = dataset_root_path + category + '/' + lang
        
        # If you wish to preprocess all data use the following command
        audio_files[lang + '.' + category] = glob(dataset_path + '/*.mp3') + glob(dataset_path + '/*.wav')
        
        # If you only wish to preprocess augmented data use the following command
        #audio_files[lang + '.' + category] = glob(dataset_path + '/*.wav')

print(audio_files.keys())

dict_keys(['english.train', 'english.test', 'german.train', 'german.test', 'swedish.train', 'swedish.test'])


## Example

In [8]:
key = list(audio_files.keys())[0]
audio_file = audio_files[key][0]
audio_file

'../data/train/english\\common_voice_en_104678.mp3'

In [17]:
audio = load_audio_file(audio_file)

#librosa.display.waveplot(audio, sr=sample_rate)
#plt.show()

In [18]:
audio_fixed = fix_audio_segment_to_10_seconds(audio)

#waveplot(audio_fixed, sr=sample_rate)
#plt.show()

In [19]:
spectro = spectrogram(audio_fixed)

plt.imshow(spectro, origin='lower', aspect='auto')
plt.show()

C:\Users\filip\AppData\Local\Temp/ipykernel_12024/1900841870.py:4: FutureWarning: Pass y=[0.         0.         0.         ... 0.00091493 0.00118123 0.00122672] as keyword args. From version 0.10 passing these as positional arguments will result in an error
  spec = lr.feature.melspectrogram(audio_segment, n_mels=image_height, hop_length=int(hl))


In [20]:
spectro.shape

(128, 500)

## Parse all Audio Files

The following script will parse all audio files to extract mel-scaled spectrogram images and store them as PNG

In [21]:
def audio_to_image_file(audio_file):
    out_image_file = audio_file + '.png'
    audio = load_audio_file(audio_file)
    audio_fixed = fix_audio_segment_to_10_seconds(audio)
    if np.count_nonzero(audio_fixed) != 0:
        spectro = spectrogram(audio_fixed)
        spectro_int = to_integer(spectro)
        imageio.imwrite(out_image_file, spectro_int)
    else:
        print('WARNING! Detected an empty audio signal. Skipping...')

In [22]:
for lang in languages:
    for category in categories:
        all_audio_files = audio_files[lang + '.' + category]
        
        num_files = len(all_audio_files)
        
        for i in range(num_files):
            if i % (num_files / 50) == 0:
                print('Still processing ' + lang + ' ' + category + ' ' + str(i) + '/' + str(num_files))
            audio_to_image_file(all_audio_files[i])

Still processing english train 0/928


C:\Users\filip\AppData\Local\Temp/ipykernel_12024/1900841870.py:4: FutureWarning: Pass y=[0.         0.         0.         ... 0.00091493 0.00118123 0.00122672] as keyword args. From version 0.10 passing these as positional arguments will result in an error
  spec = lr.feature.melspectrogram(audio_segment, n_mels=image_height, hop_length=int(hl))
C:\Users\filip\AppData\Local\Temp/ipykernel_12024/1900841870.py:4: FutureWarning: Pass y=[0.         0.         0.         ... 0.02940613 0.0360517  0.01026881] as keyword args. From version 0.10 passing these as positional arguments will result in an error
  spec = lr.feature.melspectrogram(audio_segment, n_mels=image_height, hop_length=int(hl))
C:\Users\filip\AppData\Local\Temp/ipykernel_12024/1900841870.py:4: FutureWarning: Pass y=[ 0.          0.          0.         ... -0.05684178 -0.06610166
 -0.00543618] as keyword args. From version 0.10 passing these as positional arguments will result in an error
  spec = lr.feature.melspectrogram(au

C:\Users\filip\AppData\Local\Temp/ipykernel_12024/1900841870.py:4: FutureWarning: Pass y=[0.         0.         0.         ... 0.0330749  0.03070624 0.02842894] as keyword args. From version 0.10 passing these as positional arguments will result in an error
  spec = lr.feature.melspectrogram(audio_segment, n_mels=image_height, hop_length=int(hl))
C:\Users\filip\AppData\Local\Temp/ipykernel_12024/1900841870.py:4: FutureWarning: Pass y=[ 0.          0.          0.         ... -0.02324533 -0.02217115
 -0.02172499] as keyword args. From version 0.10 passing these as positional arguments will result in an error
  spec = lr.feature.melspectrogram(audio_segment, n_mels=image_height, hop_length=int(hl))
C:\Users\filip\AppData\Local\Temp/ipykernel_12024/1900841870.py:4: FutureWarning: Pass y=[ 0.          0.          0.         ... -0.00504427  0.0041731
 -0.00741918] as keyword args. From version 0.10 passing these as positional arguments will result in an error
  spec = lr.feature.melspectrog

C:\Users\filip\AppData\Local\Temp/ipykernel_12024/1900841870.py:4: FutureWarning: Pass y=[ 0.          0.          0.         ... -0.01357094  0.0389216
  0.02851176] as keyword args. From version 0.10 passing these as positional arguments will result in an error
  spec = lr.feature.melspectrogram(audio_segment, n_mels=image_height, hop_length=int(hl))
C:\Users\filip\AppData\Local\Temp/ipykernel_12024/1900841870.py:4: FutureWarning: Pass y=[ 0.          0.          0.         ... -0.02049125  0.00108033
 -0.00789303] as keyword args. From version 0.10 passing these as positional arguments will result in an error
  spec = lr.feature.melspectrogram(audio_segment, n_mels=image_height, hop_length=int(hl))
C:\Users\filip\AppData\Local\Temp/ipykernel_12024/1900841870.py:4: FutureWarning: Pass y=[0.0000000e+00 0.0000000e+00 0.0000000e+00 ... 1.0249686e-06 5.7829839e-06
 3.1352760e-05] as keyword args. From version 0.10 passing these as positional arguments will result in an error
  spec = lr.

C:\Users\filip\AppData\Local\Temp/ipykernel_12024/1900841870.py:4: FutureWarning: Pass y=[ 0.00000000e+00  0.00000000e+00  0.00000000e+00 ...  2.83836198e-05
  1.05159515e-05 -2.91785327e-05] as keyword args. From version 0.10 passing these as positional arguments will result in an error
  spec = lr.feature.melspectrogram(audio_segment, n_mels=image_height, hop_length=int(hl))
C:\Users\filip\AppData\Local\Temp/ipykernel_12024/1900841870.py:4: FutureWarning: Pass y=[ 0.0000000e+00  0.0000000e+00  0.0000000e+00 ... -3.6431593e-04
 -1.6843011e-04  2.8716100e-05] as keyword args. From version 0.10 passing these as positional arguments will result in an error
  spec = lr.feature.melspectrogram(audio_segment, n_mels=image_height, hop_length=int(hl))
C:\Users\filip\AppData\Local\Temp/ipykernel_12024/1900841870.py:4: FutureWarning: Pass y=[0.         0.         0.         ... 0.09315857 0.06868387 0.02862575] as keyword args. From version 0.10 passing these as positional arguments will result 

...

C:\Users\filip\AppData\Local\Temp/ipykernel_12024/1900841870.py:4: FutureWarning: Pass y=[ 0.          0.          0.         ... -0.01422367 -0.01479897
 -0.00804472] as keyword args. From version 0.10 passing these as positional arguments will result in an error
  spec = lr.feature.melspectrogram(audio_segment, n_mels=image_height, hop_length=int(hl))
C:\Users\filip\AppData\Local\Temp/ipykernel_12024/1900841870.py:4: FutureWarning: Pass y=[ 0.          0.          0.         ... -0.00117225  0.00109861
 -0.00175583] as keyword args. From version 0.10 passing these as positional arguments will result in an error
  spec = lr.feature.melspectrogram(audio_segment, n_mels=image_height, hop_length=int(hl))
C:\Users\filip\AppData\Local\Temp/ipykernel_12024/1900841870.py:4: FutureWarning: Pass y=[0.         0.         0.         ... 0.01937274 0.01318875 0.0239336 ] as keyword args. From version 0.10 passing these as positional arguments will result in an error
  spec = lr.feature.melspectro

C:\Users\filip\AppData\Local\Temp/ipykernel_12024/1900841870.py:4: FutureWarning: Pass y=[ 0.          0.          0.         ... -0.18627489 -0.19310813
 -0.20412348] as keyword args. From version 0.10 passing these as positional arguments will result in an error
  spec = lr.feature.melspectrogram(audio_segment, n_mels=image_height, hop_length=int(hl))
C:\Users\filip\AppData\Local\Temp/ipykernel_12024/1900841870.py:4: FutureWarning: Pass y=[ 0.00000000e+00  0.00000000e+00  0.00000000e+00 ... -1.32043686e-04
 -1.13183705e-05  2.51759688e-04] as keyword args. From version 0.10 passing these as positional arguments will result in an error
  spec = lr.feature.melspectrogram(audio_segment, n_mels=image_height, hop_length=int(hl))
C:\Users\filip\AppData\Local\Temp/ipykernel_12024/1900841870.py:4: FutureWarning: Pass y=[ 0.          0.          0.         ... -0.09598    -0.0874408
 -0.14720933] as keyword args. From version 0.10 passing these as positional arguments will result in an error


C:\Users\filip\AppData\Local\Temp/ipykernel_12024/1900841870.py:4: FutureWarning: Pass y=[ 0.0000000e+00  0.0000000e+00  0.0000000e+00 ...  2.3520442e-06
 -1.5327638e-05  7.7996492e-06] as keyword args. From version 0.10 passing these as positional arguments will result in an error
  spec = lr.feature.melspectrogram(audio_segment, n_mels=image_height, hop_length=int(hl))
C:\Users\filip\AppData\Local\Temp/ipykernel_12024/1900841870.py:4: FutureWarning: Pass y=[ 0.          0.          0.         ... -0.28243637 -0.37651536
 -0.41818255] as keyword args. From version 0.10 passing these as positional arguments will result in an error
  spec = lr.feature.melspectrogram(audio_segment, n_mels=image_height, hop_length=int(hl))
C:\Users\filip\AppData\Local\Temp/ipykernel_12024/1900841870.py:4: FutureWarning: Pass y=[ 0.          0.          0.         ... -0.04286542 -0.02923719
 -0.02662243] as keyword args. From version 0.10 passing these as positional arguments will result in an error
  spe

C:\Users\filip\AppData\Local\Temp/ipykernel_12024/1900841870.py:4: FutureWarning: Pass y=[0.         0.         0.         ... 0.02231198 0.02316193 0.03410223] as keyword args. From version 0.10 passing these as positional arguments will result in an error
  spec = lr.feature.melspectrogram(audio_segment, n_mels=image_height, hop_length=int(hl))
C:\Users\filip\AppData\Local\Temp/ipykernel_12024/1900841870.py:4: FutureWarning: Pass y=[0.         0.         0.         ... 0.01961915 0.02248069 0.02381852] as keyword args. From version 0.10 passing these as positional arguments will result in an error
  spec = lr.feature.melspectrogram(audio_segment, n_mels=image_height, hop_length=int(hl))
C:\Users\filip\AppData\Local\Temp/ipykernel_12024/1900841870.py:4: FutureWarning: Pass y=[ 0.          0.          0.         ... -0.16521692 -0.17096047
 -0.16539223] as keyword args. From version 0.10 passing these as positional arguments will result in an error
  spec = lr.feature.melspectrogram(au

C:\Users\filip\AppData\Local\Temp/ipykernel_12024/1900841870.py:4: FutureWarning: Pass y=[ 0.          0.          0.         ... -0.01690882 -0.01530069
  0.00133438] as keyword args. From version 0.10 passing these as positional arguments will result in an error
  spec = lr.feature.melspectrogram(audio_segment, n_mels=image_height, hop_length=int(hl))
C:\Users\filip\AppData\Local\Temp/ipykernel_12024/1900841870.py:4: FutureWarning: Pass y=[ 0.          0.          0.         ...  0.00832037  0.00445405
 -0.00082906] as keyword args. From version 0.10 passing these as positional arguments will result in an error
  spec = lr.feature.melspectrogram(audio_segment, n_mels=image_height, hop_length=int(hl))
C:\Users\filip\AppData\Local\Temp/ipykernel_12024/1900841870.py:4: FutureWarning: Pass y=[ 0.          0.          0.         ...  0.03518621 -0.02090937
 -0.04736945] as keyword args. From version 0.10 passing these as positional arguments will result in an error
  spec = lr.feature.mel

## Show an Example Image

This script loads a file from disk and displays it.

In [23]:
list_of_image_files = glob(dataset_root_path + test + '/' + eng + '/*.png')
image_file_path = list_of_image_files[0]

print(image_file_path)

image = imageio.imread(image_file_path)

plt.imshow(image, origin='lower', aspect='auto')
plt.show()

print(image.shape)

../data/test/english\common_voice_en_10104.mp3.png
(128, 500)


Listen to the corresponding audio file:

In [24]:
audio_file_path = os.path.splitext(image_file_path)[0]

Audio(audio_file_path)